In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
import time
from log import get_logger  # Import your existing logger
import logging

In [2]:
# Initialize your logger
logger = get_logger(__name__)

# Suppress matplotlib warnings in logs
logging.getLogger("matplotlib").setLevel(logging.ERROR)

In [ ]:
# === Paths ===
GRAPH_SAVE_DIR = Path("/Users/ashishsingh/Desktop/Sign01/graph")
GRAPH_SAVE_DIR.mkdir(parents=True, exist_ok=True)
MODEL_PATH='/Users/ashishsingh/Desktop/Sign01/models/best_model.p'
LABELS_PATH='/Users/ashishsingh/Desktop/Sign01/models/labels.p'

In [4]:
def load_model_and_labels(model_path=MODEL_PATH, labels_path=LABELS_PATH):
    """Load trained model and label encoder."""
    logger = get_logger(__name__)
    try:
        model = pickle.load(open(model_path, 'rb'))['model'] ##if the model saved as .keras then ##model = load_model("best_model.h5")
        label_encoder = pickle.load(open(labels_path, 'rb'))
        logger.info("Model and labels loaded successfully.")
        return model, label_encoder
    except Exception as e:
        logger.error(f"Error loading model or labels: {e}")
        raise

In [5]:
def initialize_camera():
    """Initialize webcam."""
    logger = get_logger(__name__)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        logger.error("Unable to access webcam.")
        raise RuntimeError("Unable to access webcam.")
    logger.info("Webcam initialized successfully.")
    return cap

In [6]:
def initialize_mediapipe():
    """Initialize MediaPipe Hands."""
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    return hands, mp_hands, mp_drawing, mp_drawing_styles

In [7]:
def process_frame(frame, hands, mp_hands, mp_drawing, mp_drawing_styles):
    """Extract landmarks and draw them on frame."""
    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    data_aux = [0] * 84  # 2 hands × 21 landmarks × 2 coords
    bbox_coords = None

    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
            x_, y_, hand_data = [], [], []
            for landmark in hand_landmarks.landmark:
                x_.append(landmark.x)
                y_.append(landmark.y)

            for landmark in hand_landmarks.landmark:
                hand_data.append(landmark.x - min(x_))
                hand_data.append(landmark.y - min(y_))

            start_idx = i * 42
            data_aux[start_idx:start_idx + 42] = hand_data

            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            x1, y1 = int(min(x_) * W) - 10, int(min(y_) * H) - 10
            x2, y2 = int(max(x_) * W) + 10, int(max(y_) * H) + 10
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            bbox_coords = (x1, y1)

    return frame, data_aux, bbox_coords


In [8]:
def predict_and_display(data_aux, model, label_encoder, frame, bbox_coords):
    """Make prediction and draw it on the frame."""
    if bbox_coords is None or not data_aux:
        return None

    try:
        # Ensure 2D array shape: (1, n_features)
        features = np.array(data_aux).reshape(1, -1)
        
        prediction = model.predict(features)
        predicted_label = label_encoder.inverse_transform(prediction)[0]

        x1, y1 = bbox_coords
        cv2.putText(
            frame, predicted_label, (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA
        )

        return predicted_label

    except Exception as e:
        print(f"[Prediction Error] {e}")
        return None


In [9]:
def run_prediction_loop():
    logger = get_logger(__name__)
    model, label_encoder = load_model_and_labels()
    cap = initialize_camera()
    hands, mp_hands, mp_drawing, mp_drawing_styles = initialize_mediapipe()

    predictions_counter = Counter()

    plt.ion()  # Interactive mode for live graph
    start_time = time.time()

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                logger.error("Failed to read frame from webcam.")
                break

            frame, data_aux, bbox_coords = process_frame(frame, hands, mp_hands, mp_drawing, mp_drawing_styles)
            predicted_label = predict_and_display(data_aux, model, label_encoder, frame, bbox_coords)

            cv2.imshow('Sign Language Classifier', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        logger.error(f"Error in prediction loop: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        plt.ioff()
        logger.info("Prediction loop ended.")




In [10]:
# --------------------- Script Entry --------------------- #

if __name__ == "__main__":
    run_prediction_loop()


2025-09-27 10:44:25,721 | INFO     | __main__ | Model and labels loaded successfully.
2025-09-27 10:44:27,177 | INFO     | __main__ | Webcam initialized successfully.
I0000 00:00:1758950067.211650  155733 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1758950067.222043  156201 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758950067.228414  156196 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758950067.277363  156196 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-09-27 10:46:39,629 | INFO     | __main__ | Prediction loop ended.
